In [ ]:
!pip install torch
!pip install torchvision

In [3]:
import torch
import torch.nn as nn
from torchvision import models

In [7]:
model = models.resnet18(pretrained=True)

for param in model.parameters():
    param.requires_grad = False
num_classes = 29
model.fc = nn.Linear(model.fc.in_features, num_classes)

for param in model.fc.parameters():
    param.requires_grad = True

In [ ]:
print(model)


In [9]:
import torch.optim as optim

learning_rate = 0.001
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

In [10]:
early_stopping_patience = 5
best_val_loss = float("inf")
epochs_no_improve = 0


In [ ]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(root='path_to_data/train', transform=transform)
val_dataset = datasets.ImageFolder(root='path_to_data/val', transform=transform)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
import torch.nn.functional as F

# Number of epochs
num_epochs = 20

# Training loop
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    
    for images, labels in train_loader:

        images, labels = images.to('cuda'), labels.to('cuda')

        optimizer.zero_grad()

        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)

        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()

    train_loss /= len(train_loader)
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to('cuda'), labels.to('cuda')
            
            outputs = model(images)
            loss = F.cross_entropy(outputs, labels)
            val_loss += loss.item()
    
    val_loss /= len(val_loader)

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

    scheduler.step(val_loss)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= early_stopping_patience:
            print("Early stopping triggered.")
            break
